In [0]:
import pandas as pd
import numpy as np
dt=pd.read_csv(r'train.csv')

y=dt['Severity']

x=dt.drop(['Accident_ID','Severity'],axis=1)
j=pd.get_dummies(x['Accident_Type_Code'])


x=pd.concat([x,j],axis=1)


x=x.drop(['Accident_Type_Code'],axis=1)
x=pd.concat([x[['Safety_Score']]**5,x[['Safety_Score']]**4,x[['Safety_Score']]**3,x[['Safety_Score']]**2,x],axis=1)
#x=pd.concat([x[['Violations']]**4,x[['Violations']]**3,x[['Violations']]**2,x],axis=1)
#x=pd.concat([x[['Max_Elevation']]**2,x],axis=1)
x.shape


from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder




label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(y)

onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
y=integer_encoded



from sklearn.model_selection import train_test_split
x_tr,x_ts,y_tr,y_ts=train_test_split(x,y,test_size=0.2)
from sklearn.preprocessing import RobustScaler
#x_tr=RobustScaler().fit(x_tr).transform(x_tr)
#x_ts=RobustScaler().fit(x_ts).transform(x_ts)




from sklearn.preprocessing import StandardScaler  # doctest: +SKIP
scaler = StandardScaler()  # doctest: +SKIP

scaler.fit(x)  # doctest: +SKIP
x_tr = scaler.transform(x_tr)
x_ts = scaler.transform(x_ts)


In [4]:
from sklearn.tree import DecisionTreeClassifier 
dtree_model = DecisionTreeClassifier(criterion='entropy',max_depth=200,random_state=10,min_samples_split=2) 
dtree_model.fit(x_tr,y_tr)
dtree_model.score(x_ts,y_ts)

0.9275

In [0]:

from sklearn.model_selection import GridSearchCV

def dtree_grid_search(X,y,nfolds):
    #create a dictionary of all values we want to test
    param_grid = { 'criterion':['entropy'],'splitter':['best', 'random'],'max_depth': [10,15,50,150,None],'min_samples_leaf': [1,2,3, 4, 5],
    'min_samples_split': [2,4,8, 10, 12],'max_features': [None,'log2',2, 3,'auto','sqrt'],'random_state':[0,5,10,2,None]}
    # decision tree model
    dtree_model=DecisionTreeClassifier()
    #use gridsearch to test all values
    dtree_gscv = GridSearchCV(dtree_model, param_grid, cv=nfolds,n_jobs=-1)
    #fit model to data
    dtree_gscv.fit(X, y)
    print(dtree_gscv.score(x_ts, y_ts))
    print(dtree_gscv.best_params_)

In [6]:
dtree_grid_search(x_ts,y_ts,5)


0.937
{'criterion': 'entropy', 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 4, 'min_samples_split': 12, 'random_state': 0, 'splitter': 'best'}
